<a href="https://colab.research.google.com/github/Shellyga/Adversarial-Domain-Adaptation-with-Keras/blob/master/new_implementation_adda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load images

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
SEED = 7
import os
import sys
import argparse
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from plotnine import *
import pandas as pd
                
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from math import ceil
from sklearn.utils import shuffle as skshuffle

from imutils import paths
from sklearn.preprocessing import LabelEncoder

# os.environ['PYTHONHASHSEED']=str(SEED)
# np.random.seed(SEED)
# set_random_seed(SEED)
# random.seed(SEED)

from PIL import Image
from keras.utils import to_categorical
from keras.layers import Input
from keras.optimizers import Adam
from keras.utils import multi_gpu_model
from sklearn.metrics import accuracy_score
import pickle

NUM_CLASSES = 0


def pil_loader(path):
    # print(path)
    # Return the RGB variant of input image
    with open(path, 'rb') as f:
      with Image.open(f) as img:
        return img.convert('RGB')

def one_hot_encoding(param,labels):
    lb = LabelEncoder()
    fit_labels  = lb.fit_transform(labels)
    param["number_of_classe"] = len(lb.classes_) 
    labels = to_categorical(fit_labels)
    f = open('drive/My Drive/pro_data/result_vgg/labels',"wb")
    f.write(pickle.dumps(lb))
    f.close()
    return labels

            
def data_loader(filepath, inp_dims):
    # Load images and corresponding labels from the text file, stack them in numpy arrays and return
    # if not os.path.isfile(filepath):
    #     print("File path {} does not exist. Exiting...".format(filepath))
        # sys.exit() 
    img = []
    label = []
    detector = dlib.get_frontal_face_detector()
    predictor = "drive/My Drive/pro_data/shape_predictor_5_face_landmarks.dat"
    # grab the image paths
    imagePaths = sorted(list(paths.list_images(filepath)))
    # print(imagePaths)
    # loop over the input images
    for imagePath in imagePaths:
        # extract the class label from the image path and update the
        # labels list
        l = imagePath.split(os.path.sep)[-2]
        label.append(l)
        i = pil_loader(imagePath)

        i = i.resize((inp_dims[0], inp_dims[1]), Image.ANTIALIAS)
        frame_1 = align_and_crop(detector, np.array(i), predictor)[0]
        frame_1 = cv2.resize(frame_1,(inp_dims[0], inp_dims[1]))
        img.append(frame_1)
    plt.imshow(i)
    plt.imshow(frame_1)
    plt.show()
    img, label = skshuffle(img, label)
    img = np.array(img)
    label = np.array(label)
    return img, label

def get_dataset(param,path):
      print("[INFO] loading images...")
      # Load source and target data
      X,y = data_loader(path, [224,224,3])
      print("images loaded")
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
      # Encode labels into one-hot format
      print("[INFO] Encode labels into one-hot format")
      y_train = one_hot_encoding(param,y_train)
      y_test = one_hot_encoding(param,y_test)
      return (X_train, y_train), (X_test, y_test)

# Crop

In [3]:
import dlib

import os
import sys
import cv2
import glob


def align_and_crop(detector, im_to_align, predictor_path):
    '''
    simply aligns the photo (using the detector to identify the eyes) and crops the face.
    :param im_to_align: an address of an image
    '''

    # a shape predictor to find face landmarks so we can precisely localize the face
    sp = dlib.shape_predictor(predictor_path)

    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time.
    try:
        dets = detector(im_to_align, 1)
    except RuntimeError:
        return -1

    num_faces = len(dets)
    if num_faces == 0:
        return im_to_align, False

    # Find the 5 face landmarks we need to do the alignment.
    faces = dlib.full_object_detections()
    for detection in dets:
        faces.append(sp(im_to_align, detection))

    # get a single chip (aligned and cropped)
    image = dlib.get_face_chip(im_to_align, faces[0])
    # cv2.imshow("f", image) show image for testing
    return image, True

# optimizer

In [4]:
import numpy as np
from keras.optimizers import Adam

def opt_classifier(param):
    return Adam(lr=param["lr_classifier"], beta_1=param["b1_classifier"], beta_2=param["b2_classifier"])

def opt_discriminator(param):
    return Adam(lr=param["lr_discriminator"], beta_1=param["b1_discriminator"], beta_2=param["b2_discriminator"])



# ADDA

In [5]:
pip install keras_vggface

In [6]:
pip install keras_applications

In [7]:

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, clone_model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

import keras
from keras.utils import np_utils

import tensorflow.compat.v1 as tf
# from datasets import get_dataset

import matplotlib.pyplot as plt

import sys
import os
import numpy as np
import argparse
from keras_vggface.vggface import VGGFace

In [8]:
def define_source_encoder():

        inp = Input(shape= param["inp_dims"])
        network = eval('VGGFace')
        base = network(weights = 'vggface', include_top = False)
        param["source_encoder"] = Model(inp,base(inp))

def define_target_encoder():
       param["target_encoder"] = clone_model(param["source_encoder"])

def get_source_classifier(model):
        
    x = Flatten()(model.output)
    dense1 = Dense(400, name = 'class_dense1')(x)
    bn1 = BatchNormalization(name = 'class_bn1')(dense1)
    act1 = Activation('relu', name = 'class_act1')(bn1)
    drop2 = Dropout(param["drop_classifier"], name = 'class_drop1')(act1)

    dense2 = Dense(100, name = 'class_dense2')(drop2)
    bn2 = BatchNormalization(name = 'class_bn2')(dense2)
    act2 = Activation('relu', name = 'class_act2')(bn2)
    drop2 = Dropout(param["drop_classifier"], name = 'class_drop2')(act2)
    densel = Dense(param["number_of_classe"], name = 'class_dense_last')(drop2)
    # densel = Dense(param["number_of_classe"], name = 'class_dense_last')(x)
    bnl = BatchNormalization(name = 'class_bn_last')(densel)
    actl = Activation('softmax', name = 'class_act_last')(bnl)
        
    source_classifier_model = Model(inputs=(model.input), outputs=(actl))
        
    return source_classifier_model

def define_discriminator(shape):
        
    inp = Input(shape=shape)
    x = Flatten()(inp)
    dense1 = Dense(400, name = 'dis_dense1')(x)
    bn1 = BatchNormalization(name='dis_bn1')(dense1)
    act1 = Activation('relu', name = 'dis_act1')(bn1)
    drop1 = Dropout(param["drop_discriminator"], name = 'dis_drop1')(act1)

    dense2 = Dense(100, name = 'dis_dense2')(drop1)
    bn2 = BatchNormalization(name='dis_bn2')(dense2)
    act2 = Activation('relu', name = 'dis_act2')(bn2)
    drop2 = Dropout(param["drop_discriminator"], name = 'dis_drop2')(act2)

    densel = Dense(1, name = 'dis_dense_last')(drop2)
    # densel = Dense(1, name = 'dis_dense_last')(x)
    bnl = BatchNormalization(name = 'dis_bn_last')(densel)
    actl = Activation('sigmoid', name = 'dis_act_last')(bnl)
    param["discriminator_model"] = Model(inputs=(inp), outputs=(actl), name='discriminator')

def get_discriminator(model):    
    disc = Model(inputs=(model.input), outputs=(param["discriminator_model"](model.output)))
    return disc

  

In [9]:
def train_source_model(param, epochs=30, batch_size=128, save_interval=1, start_epoch=0):
        checkpoint = ModelCheckpoint("drive/My Drive/pro_data/pro_book_res/source_model.h5", monitor='val_accuracy', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto')
        early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

        (train_x, train_y), (test_x, test_y) = param["source_data_train"], param["source_data_test"]
        datagen = ImageDataGenerator(data_format='channels_last', 
                                rescale=1./255, 
                                rotation_range=40, 
                                width_shift_range=0.2, 
                                height_shift_range=0.2)
       
        evalgen = ImageDataGenerator(data_format='channels_last', 
                                rescale=1./255)
        
        param["source_model"].compile(optimizer = opt_classifier(param), loss = 'categorical_crossentropy', metrics = ['accuracy'])
        train_generator  = datagen.flow(train_x, train_y, batch_size=batch_size, shuffle=True)
        # plt.imshow(train_generator.next()[0][0])
        # plt.show()
        param["source_model"].fit_generator(train_generator ,
                            steps_per_epoch=10, 
                            epochs=epochs, 
                            validation_data=evalgen.flow(test_x, test_y, batch_size=batch_size), 
                            initial_epoch=start_epoch,callbacks=[checkpoint,early])
        
        # param["source_model"].save('drive/My Drive/pro_data/result_vgg/source_model_updated.h5')

        y_pred = param["source_model"].predict(test_x)
        source_accuracy_test = accuracy_score(test_y.argmax(1), y_pred.argmax(1))              

        print("source accuracy test",source_accuracy_test)
        print("Source Classifier matrix: ",metrics.confusion_matrix(test_y.argmax(1), y_pred.argmax(1)))

In [16]:
def train_target_discriminator(param, epochs=2, batch_size=128, save_interval=40, start_epoch=0, num_batches=120):   
    
        (source_x, _), (_,_) = param["source_data_train"], param["source_data_test"]
        
        src_datagen = ImageDataGenerator(data_format='channels_last', 
                                rescale=1./255, 
                                rotation_range=40, 
                                width_shift_range=0.2, 
                                height_shift_range=0.2)
        
        (target_x, _), (_,_) = param["target_data_train"], param["target_data_test"]
        
        tgt_datagen = ImageDataGenerator(data_format='channels_last', 
                                rescale=1./255, 
                                rotation_range=40, 
                                width_shift_range=0.2, 
                                height_shift_range=0.2)
        
        # define_source_encoder()
        # define_target_encoder()
        param["source_encoder"].save_weights('drive/My Drive/pro_data/pro_book_res/encoder_s_weights.hdf5')
        define_discriminator(param["source_encoder"].output_shape[1:])
        
        param["source_dicriminator"] = get_discriminator(param["source_encoder"])
        param["target_dicriminator"]= get_discriminator(param["target_encoder"])
        param["target_dicriminator"].compile(optimizer = opt_discriminator(param), loss = 'binary_crossentropy', metrics = ['accuracy'])

        for layer in param["source_encoder"].layers:
            layer.trainable = False
        
        param["source_dicriminator"].compile(optimizer = opt_discriminator(param), loss = 'binary_crossentropy', metrics = ['accuracy'])
        
        src_names = ['src_discriminator_loss', 'src_discriminator_acc']
        tgt_names = ['tgt_discriminator_loss', 'tgt_discriminator_acc']
        
        for iteration in range(start_epoch, epochs):
            
            avg_loss, avg_acc, index = [0, 0], [0, 0], 0

            for source,target in zip(src_datagen.flow(source_x, None, batch_size=batch_size), tgt_datagen.flow(target_x, None, batch_size=batch_size)):
                # s_domain = np_utils.to_categorical(np.zeros(source.shape[0]), 1)
                # t_domain =  np_utils.to_categorical(np.ones(target.shape[0]), 1)
                s_domain = np.zeros(source.shape[0])
                t_domain =  np.ones(target.shape[0])
                l1, acc1 = param["source_dicriminator"].train_on_batch(source,s_domain )
                l2, acc2 = param["target_dicriminator"].train_on_batch(target,t_domain)
                index+=1
                loss, acc = (l1+l2)/2, (acc1+acc2)/2
                print (iteration+1,': ', index,'/', num_batches, '; Loss: %.4f'%loss, ' (', '%.4f'%l1, '%.4f'%l2, '); Accuracy: ', acc, ' (', '%.4f'%acc1, '%.4f'%acc2, ')')
                avg_loss[0] += l1
                avg_acc[0] += acc1
                avg_loss[1] += l2
                avg_acc[1] += acc2
                if index%num_batches == 0:
                    break
            
            
                if index%save_interval==0:
                  param["source_dicriminator"].save_weights('drive/My Drive/pro_data/pro_book_res/discriminator_s_weights_%02d.hdf5'%index)
                  param["target_dicriminator"].save_weights('drive/My Drive/pro_data/pro_book_res/discriminator_t_weights_%02d.hdf5'%index)
                  param["target_encoder"].save_weights('drive/My Drive/pro_data/pro_book_res/encoder_t_weights_%02d.hdf5'%index)


In [11]:
def eval_source_classifier( param,  model,batch_size=128, domain='Source'):
      
        (train_x,_), (test_x, test_y) = param["target_data_train"], param["target_data_test"]
        src_datagen = ImageDataGenerator(data_format='channels_last', 
                                        rescale=1./255)
                      
        model.compile(optimizer = opt_classifier(param), loss = 'categorical_crossentropy', metrics = ['accuracy'])


        scores = model.evaluate_generator(src_datagen.flow(test_x, test_y),10000)
        print(' %s Classifier Test loss:%.5f'%( domain, scores[0]))
        print(' %s Classifier Test accuracy:%.2f%%'%( domain, float(scores[1])*100))            
            
def eval_target_classifier(param):
        
        # param["target_encoder"].summary()
        # model = get_source_classifier(param["target_encoder"])
        model = param["source_model"]
        path_target_discriminator = 'drive/My Drive/pro_data/pro_book_res/discriminator_t_weights.hdf5'
        model.load_weights(path_target_discriminator, by_name=True)
        model.summary()
        eval_source_classifier(param,  model, domain='Target')

In [12]:
import argparse
if __name__ == "__main__":
    # Read parameter values from the console
    parser = argparse.ArgumentParser(description = 'Domain Adaptation')
    parser.add_argument('--number_of_gpus', type = int, nargs = '?', default = '1', help = "Number of gpus to run")
    parser.add_argument('--network_name', type = str, default = 'ResNet50', help = "Name of the feature extractor network")
    parser.add_argument('--dataset_name', type = str, default = 'Office', help = "Name of the source dataset")
    parser.add_argument('--dropout_classifier', type = float, default = 0.25, help = "Dropout ratio for classifier")
    parser.add_argument('--dropout_discriminator', type = float, default = 0.25, help = "Dropout ratio for discriminator")    
    parser.add_argument('--source_path', type = str, default = 'amazon_10_list.txt', help = "Path to source dataset")
    parser.add_argument('--target_path', type = str, default = 'webcam_10_list.txt', help = "Path to target dataset")
    parser.add_argument('--lr_classifier', type = float, default = 0.0001, help = "Learning rate for classifier model")
    parser.add_argument('--b1_classifier', type = float, default = 0.9, help = "Exponential decay rate of first moment \
                                                                                             for classifier model optimizer")
    parser.add_argument('--b2_classifier', type = float, default = 0.999, help = "Exponential decay rate of second moment \
                                                                                            for classifier model optimizer")
    parser.add_argument('--lr_discriminator', type = float, default = 0.00001, help = "Learning rate for discriminator model")
    parser.add_argument('--b1_discriminator', type = float, default = 0.9, help = "Exponential decay rate of first moment \
                                                                                             for discriminator model optimizer")
    parser.add_argument('--b2_discriminator', type = float, default = 0.999, help = "Exponential decay rate of second moment \
                                                                                            for discriminator model optimizer")
    parser.add_argument('--lr_combined', type = float, default = 0.00001, help = "Learning rate for combined model")
    parser.add_argument('--b1_combined', type = float, default = 0.9, help = "Exponential decay rate of first moment \
                                                                                             for combined model optimizer")
    parser.add_argument('--b2_combined', type = float, default = 0.999, help = "Exponential decay rate of second moment \
                                                                                            for combined model optimizer")
    parser.add_argument('--classifier_loss_weight', type = float, default = 1, help = "Classifier loss weight")
    parser.add_argument('--discriminator_loss_weight', type = float, default = 4, help = "Discriminator loss weight")
    parser.add_argument('--batch_size', type = int, default = 32, help = "Batch size for training")
    parser.add_argument('--test_interval', type = int, default = 3, help = "Gap between two successive test phases")
    parser.add_argument('--num_iterations', type = int, default = 120, help = "Number of iterations")
    parser.add_argument('--snapshot_interval', type = int, default = 500, help = "Minimum gap between saving outputs")
    parser.add_argument('--output_dir', type = str, default = 'Models', help = "Directory for saving outputs")
    # args = parser.parse_args()

    # Set GPU device
    # os.environ["CUDA_VISIBLE_DEVICES"] = str(list(np.arange(args.number_of_gpus))).strip('[]')

    # Initialize parameters
    param = {}
    param["number_of_gpus"] = 1
    # param["network_name"] = 'ResNet50'
    param["network_name"] = 'VGGFace'
    param["inp_dims"] = [224, 224, 3]
    param["num_iterations"] = 1000
    # param["num_iterations"] = 500
    param["lr_classifier"] = 0.0001
    param["b1_classifier"] = 0.9
    param["b2_classifier"] = 0.999    
    param["lr_discriminator"] = 0.00001
    param["b1_discriminator"] =  0.9
    param["b2_discriminator"] = 0.999
    param["lr_combined"] = 0.00001
    param["b1_combined"] =  0.9
    param["b2_combined"] =  0.999       
    # param["batch_size"] = int(32)
    param["batch_size"] = int(32/2)
    param["class_loss_weight"] = 1
    param["dis_loss_weight"] = 4    
    param["drop_classifier"] = 0.25
    param["drop_discriminator"] = 0.25
    param["test_interval"] = 100
    # param["source_path"] = 'drive/My Drive/final_proj_dataset/data_file_shelly.txt'
    # param["target_path"] = 'drive/My Drive/final_proj_dataset/data_file_yerus.txt' 
    param["source_path"] = 'drive/My Drive/pro_data/Student_source'
    param["target_path"] = 'drive/My Drive/pro_data/alyn_target' 
    param["snapshot_interval"] = 500
    # param["snapshot_interval"] = 5
    param["output_path"] = 'drive/My Drive/pro_data/pro_book_res'
    param["number_of_classe"] = 0    
    

   

In [13]:
    param["source_data_train"], param["source_data_test"] = get_dataset(param,param["source_path"])
    # param["target_data_train"], param["target_data_test"] = get_dataset(param,param["target_path"])


[INFO] loading images...
images loaded
[INFO] Encode labels into one-hot format


In [14]:
print("[INFO] loading images...")
# Load source and target data
X,y = data_loader(param["target_path"], [224,224,3])
print("images loaded")
X_test,y_test = data_loader('drive/My Drive/pro_data/testset', [224,224,3])
print("[INFO] Encode labels into one-hot format")
y = one_hot_encoding(param,y)
y_test = one_hot_encoding(param,y_test)
param["target_data_train"], param["target_data_test"] =(X, y), (X_test, y_test)

[INFO] loading images...
images loaded
[INFO] Encode labels into one-hot format


In [ ]:
plt.imshow(X_test[0])
plt.show()

In [19]:
y = one_hot_encoding(param,y)
y_test = one_hot_encoding(param,y_test)
param["target_data_train"], param["target_data_test"] =(X, y), (X_test, y_test)

In [28]:
print("number of images in target domain",X.shape[0])
print("number of images in target test domain",X_test.shape[0])
print("number of images in source domain", param["source_data_train"][0].shape[0])
print("number of images in source test domain", param["source_data_test"][0].shape[0])


number of images in target domain 1440
number of images in target test domain 40
number of images in source domain 1159
number of images in source test domain 290


In [15]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
define_source_encoder()
define_target_encoder()
param["source_model"] = get_source_classifier(param["source_encoder"])
train_source_model(param)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
10/10 [==============================] - ETA: 0s - loss: 1.0921 - accuracy: 0.4737
Epoch 00001: val_accuracy improved from -inf to 0.62759, saving model to drive/My Drive/pro_data/pro_book_res/source_model.h5
10/10 [==============================] - 21s 2s/step - loss: 1.0921 - accuracy: 0.4737 - val_loss: 1.0211 - val_accuracy: 0.6276
Epoch 2/30
10/10 [==============================] - ETA: 0s - loss: 0.7959 - accuracy: 0.6419
Epoch 00002: val_accuracy improved from 0.62759 to 0.71379, saving model to drive/My Drive/pro_data/pro_book_res/source_model.h5
10/10 [==============================] - 20s 2s/step - loss: 0.7959 - accuracy: 0.6419 - val_loss: 0.9648 - val_accuracy: 0.7138
Epoch 3/30
10/10 [==============================] - ETA: 0s - loss: 0.6674 - accuracy: 0.7204
Epoch 00003: val_accuracy improved from 0.71379 to 0.82759, saving model to drive/My Drive/pro_data/pro_book_res/source_model.h5


In [18]:
from keras.models import load_model
test_x,test_y = param["target_data_test"]
param["source_model"] = load_model('drive/My Drive/pro_data/pro_book_res/source_model.h5')

y_pred = param["source_model"].predict(test_x)
target_accuracy_test = accuracy_score(test_y.argmax(1), y_pred.argmax(1))              

print("target accuracy test",target_accuracy_test)
print("target Classifier matrix: ",metrics.confusion_matrix(test_y.argmax(1), y_pred.argmax(1)))

target accuracy test 0.225
target Classifier matrix:  [[ 9  0  0]
 [14  0  2]
 [15  0  0]]


In [19]:
eval_source_classifier( param,  param["source_model"])

Instructions for updating:
Please use Model.evaluate, which supports generators.
 Source Classifier Test loss:1.11158
 Source Classifier Test accuracy:25.00%


In [ ]:
param["source_model"].save('drive/My Drive/pro_data/result_vgg/source_model.h5')
# loaded_model = load_model('drive/My Drive/pro_data/result_vgg/model.h5')

In [20]:
# define_source_encoder()
# define_target_encoder()
train_target_discriminator(param)

1 :  1 / 120 ; Loss: 0.8038  ( 0.8057 0.8019 ); Accuracy:  0.51171875  ( 0.5625 0.4609 )
1 :  2 / 120 ; Loss: 0.8053  ( 0.8061 0.8046 ); Accuracy:  0.515625  ( 0.5156 0.5156 )
1 :  3 / 120 ; Loss: 0.8051  ( 0.8070 0.8031 ); Accuracy:  0.4921875  ( 0.5234 0.4609 )
1 :  4 / 120 ; Loss: 0.8041  ( 0.8033 0.8048 ); Accuracy:  0.4921875  ( 0.5156 0.4688 )
1 :  5 / 120 ; Loss: 0.8017  ( 0.8040 0.7995 ); Accuracy:  0.5234375  ( 0.5781 0.4688 )
1 :  6 / 120 ; Loss: 0.7975  ( 0.8042 0.7908 ); Accuracy:  0.51171875  ( 0.5547 0.4688 )
1 :  7 / 120 ; Loss: 0.7977  ( 0.8060 0.7893 ); Accuracy:  0.453125  ( 0.4844 0.4219 )
1 :  8 / 120 ; Loss: 0.8023  ( 0.8045 0.8002 ); Accuracy:  0.5  ( 0.5391 0.4609 )
1 :  9 / 120 ; Loss: 0.8018  ( 0.8006 0.8031 ); Accuracy:  0.5078125  ( 0.5625 0.4531 )
1 :  10 / 120 ; Loss: 0.8009  ( 0.8048 0.7971 ); Accuracy:  0.4525669664144516  ( 0.4286 0.4766 )
1 :  11 / 120 ; Loss: 0.7932  ( 0.8047 0.7816 ); Accuracy:  0.41796875  ( 0.4688 0.3672 )
1 :  12 / 120 ; Loss: 0.80

In [21]:
param["source_dicriminator"].save_weights('drive/My Drive/pro_data/pro_book_res/discriminator_s_weights.hdf5')
param["target_dicriminator"].save_weights('drive/My Drive/pro_data/pro_book_res/discriminator_t_weightsd.hdf5')
param["target_encoder"].save_weights('drive/My Drive/pro_data/pro_book_res/encoder_t_weights.hdf5')

In [22]:
from keras.models import load_model
# test_x,test_y = param["target_data_test"]
test_x,test_y = param["target_data_test"]
model = load_model('drive/My Drive/pro_data/pro_book_res/source_model.h5')
path_target_discriminator = 'drive/My Drive/pro_data/pro_book_res/discriminator_t_weightsd.hdf5'
model.load_weights(path_target_discriminator, by_name=True)
y_pred = model.predict(test_x)
target_accuracy_test = accuracy_score(test_y.argmax(1), y_pred.argmax(1))              

print("target accuracy test",target_accuracy_test)
print("target Classifier matrix: ",metrics.confusion_matrix(test_y.argmax(1), y_pred.argmax(1)))

target accuracy test 0.225
target Classifier matrix:  [[ 9  0  0]
 [16  0  0]
 [15  0  0]]


In [23]:
from keras.models import load_model
# test_x,test_y = param["target_data_test"]
test_x,test_y = param["source_data_test"]
# model = load_model('drive/My Drive/pro_data/pro_book_res/source_model.h5')
# path_target_discriminator = 'drive/My Drive/pro_data/pro_book_res/discriminator_t_weightsd.hdf5'
# model.load_weights(path_target_discriminator, by_name=True)
y_pred = model.predict(test_x)
target_accuracy_test = accuracy_score(test_y.argmax(1), y_pred.argmax(1))              

print("source accuracy test",target_accuracy_test)
print("source Classifier matrix: ",metrics.confusion_matrix(test_y.argmax(1), y_pred.argmax(1)))

source accuracy test 0.36551724137931035
source Classifier matrix:  [[106   0   0]
 [ 97   0   0]
 [ 87   0   0]]


In [ ]:
param["target_dicriminator"].save_weights('drive/My Drive/pro_data/pro_book_res/discriminator_t_weights_iter_200.hdf5')
param["target_encoder"].save_weights('drive/My Drive/pro_data/pro_book_res/encoder_t_weights_iter_200.hdf5')


In [ ]:
eval_target_classifier(param)

Model: "model_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
vggface_vgg16 (Model)        multiple                  14714688  
_________________________________________________________________
flatten_15 (Flatten)         (None, 25088)             0         
_________________________________________________________________
class_dense1 (Dense)         (None, 400)               10035600  
_________________________________________________________________
class_bn1 (BatchNormalizatio (None, 400)               1600      
_________________________________________________________________
class_act1 (Activation)      (None, 400)               0         
_________________________________________________________________
class_drop1 (Dropout)        (None, 400)               0  

In [ ]:
# target_classifier = get_source_classifier(param["target_encoder"])

# path_target_discriminator = 'drive/My Drive/pro_data/result_vgg/discriminator_t_weights.hdf5'
# target_classifier.load_weights(path_target_discriminator, by_name=True)
model = param["source_model"]
path_target_discriminator = 'drive/My Drive/pro_data/result_vgg/discriminator_t_weights_iter_400.hdf5'
model.load_weights(path_target_discriminator, by_name=True)
model.compile(optimizer = opt_classifier(param), loss = 'categorical_crossentropy', metrics = ['accuracy'])
test_x,test_y = param["target_data_test"]
yt_pred = model.predict(test_x)
target_accuracy_test = accuracy_score(test_y.argmax(1), yt_pred.argmax(1))              

print("target accuracy test",target_accuracy_test)
print("target Classifier matrix: ",metrics.confusion_matrix(test_y.argmax(1), yt_pred.argmax(1)))


target accuracy test 0.49061032863849763
target Classifier matrix:  [[  0 217]
 [  0 209]]


In [ ]:
define_target_encoder()
model = get_source_classifier(param["target_encoder"])
path = 'drive/My Drive/pro_data/result_vgg/source_model.h5'
target_discriminator = 'drive/My Drive/pro_data/result_vgg/discriminator_t_weights_iter_400.hdf5'
path_target_encoder = 'drive/My Drive/pro_data/result_vgg/encoder_t_weights.hdf5'
model.load_weights(path, by_name=True)
model.load_weights(path_target_encoder, by_name=True)

model.compile(optimizer = opt_classifier(param), loss = 'categorical_crossentropy', metrics = ['accuracy'])
test_x,test_y = param["target_data_test"]
yt_pred = model.predict(test_x)
target_accuracy_test = accuracy_score(test_y.argmax(1), yt_pred.argmax(1))              

print("target accuracy test",target_accuracy_test)
print("target Classifier matrix: ",metrics.confusion_matrix(test_y.argmax(1), yt_pred.argmax(1)))

target accuracy test 0.5093896713615024
target Classifier matrix:  [[217   0]
 [209   0]]


# Evaluation

In [ ]:
def eval_source_classifier(param, model):
        
        (test_x, test_y) = param["target_data_test"]
        src_datagen = ImageDataGenerator(data_format='channels_last', 
                                        rescale=1./255)
                      
        model.compile(loss='categorical_crossentropy', optimizer=self.src_optimizer, metrics=['accuracy'])

        scores = model.evaluate_generator(src_datagen.flow(test_x, test_y),10000)
        print('%s %s Classifier Test loss:%.5f'%(dataset.upper(), domain, scores[0]))
        print('%s %s Classifier Test accuracy:%.2f%%'%(dataset.upper(), domain, float(scores[1])*100))            
            
    def eval_target_classifier(param, source_model, target_discriminator, dataset='svhn'):
        source_classifier_model = load_model(source_model)
        source_classifier_model.load_weights(target_discriminator, by_name=True)
        source_classifier_model.summary()
        eval_source_classifier(param,source_classifier_model, domain='Target')

In [ ]:

# Path to source classifier model to test/evaluate
path_source_classifier = 
# Path to target discriminator model to test/evaluate
path_target_classifier= 

define_target_encoder(args.source_weights)

train_target_discriminator(param)
eval_target_classifier(path_source_classifier, path_target_classifier)

In [ ]:
model = load_model('/content/drive/My Drive/pro_data/‏final_result_notB/source_classifier.h5')
model.load_weights('/content/drive/My Drive/pro_data/pro_book_res/disc_target.hdf5', by_name=True)
